In [ ]:
# Logic to filter instrument name from company
import mysql.connector, re
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

connection = mysql.connector.connect(
    host="localhost",
    port=3307,
    user='user',
    password='userpassword',
    database='softeam'
)

cursor = connection.cursor()

def match_company_script(company_name):

    if not company_name:
        raise Exception("No company name provided")
    
    name_words = list(filter(None, re.split(r'[^a-zA-Z0-9]+', company_name)))

    name_clause = f"name LIKE '%{'%'.join(name_words)}%'"
    symbol_clauses = [f"trading_symbol LIKE '%{word}%'" for word in name_words]
    dynamic_condition = f"{name_clause} OR " + " OR ".join(symbol_clauses)

    sql_script = f"""
                    select * from upstox_instrument_data 
                    where segment in ('NSE_EQ') 
                    and ({dynamic_condition}) 
                    and instrument_type = 'EQ';
                """

    #log.info("Extracting similar matching companies from available data")
    print(sql_script)
    df = pd.read_sql_query(sql_script, connection)

    if df.empty:
        raise Exception (f"Unable to extract similarly matching details from DB for company name {company_name}")

    single_company = __apply_filtering_criteria(company_name,df)
    if single_company.empty:
        raise Exception (f"Matching strategies are not working for SQL results for company {company_name}")
    
    print(f"Best match for company {company_name} found to be {single_company['name'].iloc[0]} with trading symbol {single_company['trading_symbol'].iloc[0]}")
    return single_company


def __apply_filtering_criteria(search_term,df):

    if len(df.index) == 1:
        return df

    exact_matches = df[df['trading_symbol'].str.upper() == search_term.upper()]

    # If exact matches are found, return them
    if len(exact_matches.index) == 1:
        return exact_matches

    # If no exact matches, search for partial matches in the name or company name columns
    partial_matches = df[df['name'].str.contains(search_term, case=False) | 
                         df['short_name'].str.contains(search_term, case=False)]
    
    if len(partial_matches.index) == 1:
        return partial_matches

    return pd.DataFrame()

match_company_script('Power Grid Corporation of India Ltd.').to_dict(orient='records')[0]

In [ ]:
# Logic to filter instrument name from company
import mysql.connector, re
import pandas as pd
import warnings, traceback
from fuzzywuzzy import process
import re

warnings.filterwarnings("ignore")

ner_list = [
"SBC Communications",
"Honeywell International Inc.",
"Apple Inc.",
"Salesforce.com",
"Home Depot Inc.",
"Nvidia",
"Dow",
"Dow Inc.",
"Exxon Mobil Corp.",
"General Electric Co.",
"Nvidia Corp",
"50 Park Investments",
"Pfizer Inc.",
"Sherwin-Williams Co.",
"Advisors Asset Management",
"Intel Corp.",
"Microsoft Corp.",
"Raytheon Co.",
"Amgen Inc.",
"Apple",
"AI",
"Intel",
"Amazon.com Inc.",
"Walgreens Boots Alliance Inc.",
"Bloomberg"
]

connection = mysql.connector.connect(
    host="localhost",
    port=3307,
    user='user',
    password='userpassword',
    database='softeam'
)

cursor = connection.cursor()

def match_company_script(company_name):

    cleaned_name = remove_common_words(company_name)

    if not cleaned_name:
        raise Exception("No company name provided")

    name_words = list(filter(None, re.split(r'[^a-zA-Z0-9]+', cleaned_name)))

    name_clause = f"name LIKE '%{'%'.join(name_words)}%'"
    symbol_clauses = [f"trading_symbol LIKE '%{word}%'" for word in name_words]
    dynamic_condition = f"{name_clause} OR " + " OR ".join(symbol_clauses)

    sql_script = f"""
select trading_symbol, short_name, name, isin from upstox_instrument_data 
where segment in ('NSE_EQ') 
and ({dynamic_condition}) 
and instrument_type = 'EQ';
                """

    #log.info("Extracting similar matching companies from available data")
    print(sql_script)
    df = pd.read_sql_query(sql_script, connection)

    if df.empty:
        raise Exception (f"Unable to extract any similarly matching details from DB for company name {company_name}")

    single_company = __apply_filtering_criteria(company_name,df)
    if single_company.empty:
        raise Exception (f"Matching strategies are not working for SQL results for company {company_name}")
    
    print(f"Best match for company {company_name} found to be {single_company['name'].iloc[0]} with trading symbol {single_company['trading_symbol'].iloc[0]}")
    return single_company


def __apply_filtering_criteria(search_term,df):

    exact_matches = df[df['trading_symbol'].str.upper() == search_term.upper()]

    # If exact matches are found, return them
    if len(exact_matches.index) == 1:
        print("Exact match against trading symbol")
        return exact_matches

    best_match = process.extractOne(search_term.upper(), df['name'])
    print(f"Closest match: {best_match[0]} with a match score of {best_match[1]}")
    if best_match[1] > 90:
        return df[df['name'] == best_match[0] ]

    partial_matches = df[df['name'].str.contains(search_term, case=False) | 
                         df['short_name'].str.contains(search_term, case=False)]
    
    if len(partial_matches.index) == 1:
        return partial_matches

    print("No satisfying match found")
    return pd.DataFrame()

def remove_common_words(company_name):

    cleaned_name = company_name.lower()
    # List of common words to remove
    common_words = [
        "corporation","india","of","ltd","limited"
    ]
    cleaned_name = re.sub(r"[^a-zA-Z\s]", "", cleaned_name)
    
    # Remove each common word from the company name
    for word in common_words:
        cleaned_name = cleaned_name.replace(word, "")
    
    # Remove multiple spaces left by removed words
    cleaned_name = re.sub(r"\s+", " ", cleaned_name)
    print(f"{company_name} was cleaned to {cleaned_name}")
    return cleaned_name

#print(remove_common_words('Power Grid Corporation of India Ltd.'))
for word in ner_list:
    try:
        print(match_company_script(word).to_dict(orient='records')[0])
    except Exception as ex:
        print(f"Failed for {word} with msg -> {ex}")
        #traceback.print_exc()
    finally:
        print("~~~~~~~~~~~~~~~")

In [31]:
import csv, os

def write_to_csv(file_name, data):
    file_exists = os.path.isfile(file_name)
    with open(file_name, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames = data.keys()) 
        if not file_exists:
            writer.writeheader() 
        writer.writerow(data)

write_to_csv('test.csv', {
    'key1' : 'value1',
    'key2' : 'value2',
    'key3' : 'value3'
})
write_to_csv('test.csv', {
    'key1' : 'value1',
    'key2' : 'value2',
    'key3' : 'value3'
})
write_to_csv('test.csv', {
    'key1' : 'value1',
    'key2' : None,
    'key3' : 'value3'
})